In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import sys
sys.path.append('../src')
from data.audio_dataset import AudioSpectrogramDataset
from models.GAN import GAN

import torch
import librosa
import soundfile as sf
from torch.utils.data import DataLoader

C:\Users\Michał\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
base_directory = "../data/raw/Magenta/"

dataset = AudioSpectrogramDataset(base_directory=base_directory, spectro_type='mel-spectrogram')
dataloader = DataLoader(dataset, batch_size=16, shuffle=True, num_workers=0)

In [4]:
MusicGAN = GAN()

MusicGAN.train(dataloader=dataloader)

[Epoch 0/1] [Batch 0] [D loss: 0.6841230392456055] [G loss: 0.692092776298523]


In [5]:
MusicGAN.generator.eval()

Generator(
  (l1): Sequential(
    (0): Linear(in_features=100, out_features=704512, bias=True)
  )
  (conv_blocks): Sequential(
    (0): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (1): Upsample(scale_factor=2.0, mode=nearest)
    (2): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): BatchNorm2d(128, eps=0.8, momentum=0.1, affine=True, track_running_stats=True)
    (4): LeakyReLU(negative_slope=0.2, inplace=True)
    (5): Upsample(scale_factor=2.0, mode=nearest)
    (6): Conv2d(128, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (7): BatchNorm2d(64, eps=0.8, momentum=0.1, affine=True, track_running_stats=True)
    (8): LeakyReLU(negative_slope=0.2, inplace=True)
    (9): Conv2d(64, 1, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (10): Tanh()
  )
)

In [6]:
z = torch.randn(1, MusicGAN.latent_dim).to(MusicGAN.device)
generated_spectrogram = MusicGAN.generator(z)

In [7]:
spectrogram = generated_spectrogram.cpu().detach().numpy()
spectrogram = spectrogram.squeeze()

In [8]:
audio = librosa.feature.inverse.mel_to_audio(spectrogram)

path = '../models/generated_samples/'
file_name = 'generated_audio.wav'

sf.write(path + file_name, audio, 22050)